In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key="YOUR API KEY, 
    model_name="llama-3.1-70b-versatile"
)



In [3]:
response = llm.invoke("The first pakistani crikcter who win the worldcup ...")
print(response.content)


The first Pakistani cricketer to win the Cricket World Cup was Imran Khan, who led the Pakistani team to victory in the 1992 Cricket World Cup.


In [12]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results?q=machine+learning")
page_data = loader.load().pop().page_content
print(page_data)

Search Jobs — Google CareersCareersSkip navigation linksTeamsLocationsBenefitsJobsStudentsYour careerarrow_drop_downYour career menu ApplicationsProfileSaved jobsJob alertsSign inCareersApplicationsProfileSearch jobsSaved jobsJob alertsTeamsLocationsBenefitsJobsStudentsHelpSend FeedbackSearch JobsSearch sidebar443  jobs matchedClear filtersWhat do you want to do?Locationsexpand_moreExperienceexpand_moreSkills & qualificationsexpand_moreDegreeexpand_moreJob typesexpand_moreOrganizationsexpand_moreSort byexpand_moreFollow Life at Google on More about usAbout usopen_in_newContact usopen_in_newPressopen_in_newRelated InformationInvestor relationsopen_in_newBlogopen_in_newEqual OpportunityGoogle is proud to be an equal opportunity and affirmative action employer. We are committed to building a workforce that is representative of the users we serve, creating a culture of belonging, and providing an equal employment opportunity regardless of race, creed, color, religion, gender, sexual orient

In [13]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [14]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res


[{'role': 'Staff Machine Learning Engineer, Personal AI, Pixel',
  'experience': 'Advanced',
  'skills': 'Python, C++, data structures/algorithms, machine learning',
  'description': 'Experience owning outcomes and decision making, solving ambiguous problems and influencing stakeholders; deep expertise in domain.'},
 {'role': 'Machine Learning Acceleration Silicon Architect',
  'experience': 'Mid',
  'skills': 'Electrical Engineering, Computer Engineering, Computer Science, machine learning, deep learning principles',
  'description': 'Experience driving progress, solving problems, and mentoring more junior team members; deeper expertise and applied knowledge within relevant area.'},
 {'role': 'Cloud Engineer, AI, Google Cloud (English, Spanish)',
  'experience': 'Mid',
  'skills': 'Computer Science, machine learning solutions, technical customers, Python, Java, Go, C, C++, data structures, algorithms, software design',
  'description': 'Experience driving progress, solving problems, a

In [17]:
import pandas as pd
df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [23]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [27]:
links = collection.query(query_texts=["Experiance in python","Expertise in machine learning"] ,n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [30]:
job=json_res
job

[{'role': 'Staff Machine Learning Engineer, Personal AI, Pixel',
  'experience': 'Advanced',
  'skills': 'Python, C++, data structures/algorithms, machine learning',
  'description': 'Experience owning outcomes and decision making, solving ambiguous problems and influencing stakeholders; deep expertise in domain.'},
 {'role': 'Machine Learning Acceleration Silicon Architect',
  'experience': 'Mid',
  'skills': 'Electrical Engineering, Computer Engineering, Computer Science, machine learning, deep learning principles',
  'description': 'Experience driving progress, solving problems, and mentoring more junior team members; deeper expertise and applied knowledge within relevant area.'},
 {'role': 'Cloud Engineer, AI, Google Cloud (English, Spanish)',
  'experience': 'Mid',
  'skills': 'Computer Science, machine learning solutions, technical customers, Python, Java, Go, C, C++, data structures, algorithms, software design',
  'description': 'Experience driving progress, solving problems, a

In [32]:
prompt_email = PromptTemplate.from_template(
        """
       {job_description}
    
    ### INSTRUCTION:
    You are Javeria, a business development executive at Kickstart. Kickstart is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools. 
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
    process optimization, cost reduction, and heightened overall efficiency. 
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Kickstart 
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase Kickstart's portfolio: {link_list}
    Remember you are Javeria, BDE at Kickstart. 
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock the Power of AI and Software Solutions with Kickstart

Dear Hiring Manager,

I came across the job postings for various roles at Google, and I was impressed by the company's commitment to innovation and excellence. As a Business Development Executive at Kickstart, I believe our team can help you achieve your goals by providing top-notch AI and software solutions.

At Kickstart, we specialize in developing tailored solutions that cater to the unique needs of our clients. Our team of experts has extensive experience in machine learning, data structures, algorithms, and software development. We have a proven track record of delivering scalable, efficient, and cost-effective solutions that drive business growth.

Our portfolio showcases our capabilities in machine learning and Python development. You can explore our projects at:

https://example.com/ml-python-portfolio
https://example.com/python-portfolio

We understand that each role at Google requires a specific set of sk